# PyTorch: Tensor와 autograd

앞의 예제에서 우리는 신경망의 순전파 단계와 역전파 단계를 수동으로 구현하였습니다. 작은 2-계층 신경망에서 역전파 단계를 직접 구현하는 것은 큰 일이 아니지만, 대규모의 복잡한 신경망에서는 매우 아슬아슬한 일일 것입니다.

다행히도, 자동 미분을 사용하여 신경망에서 역전파 단계의 연산을 자동화할 수 있습니다. PyTorch의 autograd 패키지는 이 기능을 정확히 제공합니다. Autograd를 사용할 때, 신경망의 순전파 단계는 연산 그래프를 정의합니다. 그래프의 노드(Node)는 Tensor이며, 엣지(Edge)는 입력 Tensor로부터 출력 Tensor를 만들어내는 함수입니다. 이 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산할 수 있습니다.

이는 복잡하게 들리지만, 실제로 사용하는 매우 간단합니다. 각 Tensor는 연산그래프에서 노드로 표현(represent)됩니다. 만약 x가 x.requires_grad=True인 Tensor라면 x.grad는 어떤 스칼라 값에 대해 x의 변화도(gradient)를 갖는 또 다른 Tensor입니다.

여기에서는 PyTorch Tensor와 autograd를 사용하여 2-계층 신경망을 구현합니다. 이제 더 이상 신경망의 역전파 단계를 직접 구현할 필요가 없습니다.


자동 미분(Automatic differentiation) : https://en.wikipedia.org/wiki/Automatic_differentiation

수학과 컴퓨터 algebra에서 자동미분은 컴퓨터 프로그램에 의해 특정한 함수의 derivative를 수치로 평가하는 일련의 기법이다. 자동미분은 모든 컴퓨터 프로그램이 아무리 복잡하더라도 기초 산술 연산(더하기, 빼기, 곱하기, 나누기 등)과 기초 함수(exp,log,sin,cos 등)의 sequence를 실행한다는 사실을 이용한다. chain rule을 이러한 연산에 반복적으로 적용함으로써, derivatives of arbitrary order은 자동으로 계산될 수 있다. working precision에 더 정확하며 최소한 원래의 프로그램보다 더 작은 상수 계수를 사용하여 연산할 수 있다.

In [1]:
import torch

In [2]:
dtype = torch.float
#device = torch.device("cpu") cpu에서 실행
device = torch.device("cuda:0") #GPU에서 실행

In [3]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [4]:
# 입력과 출력을 저장(Hold)하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grade=False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 
# 계산할 필요가 없음을 나타냅니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [5]:
# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 
# 계산할 필요가 있음을 나타냅니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [6]:
learning_rate = 1e-6

In [7]:
for t in range(501):
    # 순전파 단계: Tensor 연산을 사용하여 y값을 예측합니다. 이는 Tensor를 사용한
    # 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로 구현하지 않아도 되므로 
    # 중간값들(Intermediate Value)에 대한 참조(Reference)를 갖고 있을 필요가 없습니다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산하고 출력합니다. 
    # loss는 (1,) 모양을 갖는 Variable이며, loss.data는 (1,) 모양의 Tensor입니다.
    # loss.data[0]은 손실(loss)의 스칼라 값입니다.
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t%20 == 0:
        print(t, loss.item())
    
    # autograde를 사용하여 역전파 단계를 계산합니다. 이는 requires_grad=True를 
    # 갖는 모든 Tensor에 대한 손실의 변화도를 계산합니다. 이후 w1.grad와 w2.grad는
    # w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 됩니다.
    loss.backward()
    
    # 경사하강법(Gradient Descent)를 사용하여 가중치를 수동으로 갱신합니다.
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True인데 
    # autograd에서는 이를 추적할 필요가 없기 때문입니다.
    # 다른 방법은 weight.data 및 weight.grad.data를 조작(Operate)하는 방법입니다.
    # tensor.data 가 tensor의 저장공간(Storage)를 공유하기는 하지만, 이력을
    # 추적하지 않는다는 것을 기억하십시오.
    # 또한, 이것을 달성하기 위해 torch.optim.SGD를 사용할 수도 있습니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        # (다음 계산에서 이전 계산도의 변화도가 영향을 주는 것을 방지) 
        w1.grad.zero_()
        w2.grad.zero_()    

0 27454140.0
20 239662.0625
40 33394.9296875
60 7889.64111328125
80 2351.244140625
100 794.6135864257812
120 290.2902526855469
140 111.4997329711914
160 44.41440200805664
180 18.177715301513672
200 7.607266902923584
220 3.24257230758667
240 1.4038186073303223
260 0.6158874034881592
280 0.2733685076236725
300 0.12257727980613708
320 0.05549684166908264
340 0.02536962553858757
360 0.011776259168982506
380 0.005606009159237146
400 0.0027802635449916124
420 0.001457701437175274
440 0.0008149440982379019
460 0.0004890320706181228
480 0.00031241323449648917
500 0.00021095119882375002


# PyTorch: 새 autograd 함수 정의하기

Under the hood, autograd의 기본(primitive) 연산자는 실제로 Tensor를 조작하는 2개의 함수입니다. **forward**함수는 입력 Tensor로부터 출력 Tensor를 계산합니다. **backward**함수는 출력 Tensor의 변화도를 받고 입력 Tensor의 변화도를 계산합니다.

PyTorch에서 **torch.autograd.Function**의 서브클래스(subclass)를 정의하고 **forward**와 **backward** 함수를 구현함으로써 쉽게 사용자 정의 autograd 연산자를 정의할 수 있습니다. 그 후, 인스턴스(instance)를 생성하고 함수처럼 호출하여 입력 데이터를 포함하는 Tensor를 전달하는 식으로 새로운 autograd 연산자를 쉽게 사용할 수 있습니다.

이 예제에서는 ReLU 비선형성(nonlinearity)을 수행하기 위한 사용자 정의 autograd 함수를 정의하고, 2-계층 신경망에 이를 적용해보도록 하겠습니다.

In [8]:
import torch

In [9]:
class MyReLU(torch.autograd.Function):
    '''
    torch.autograd.Function을 상속받아 사용자 정의 autograd 함수를 구현하고, 
    Tensor 연산을 하는 순전파와 역전파 단계를 구현하겠습니다.
    '''
    
    # @staticmethod 를 빼면 뒤에서 loss.backward를 할 때 NotImplementedError가 뜬다.
    # 왜 Error가 뜨는지는 나중에 추가 필요
    
    # @staticmethod 는 정적메소드로 클래스에서 직접 접근하며, 객체별로 달라지는 것이
    # 아니라 함께 공유하는 것이다.
    
    @staticmethod
    def forward(ctx,input):
        '''
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환합니다.
        ctx는 역전파 연산을 위한 정보를 저장하기 위해 사용하는 Context Object입니다.
        ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 어떠한 
        객체(object)도 저장(cache)해 둘 수 있습니다.
        '''
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx,grad_output):
        '''
        역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고, 입력에
        대한 손실의 변화도를 계산합니다.
        '''
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [10]:
dtype = torch.float
#device = torch.device("cpu") cpu에서 실행
device = torch.device("cuda:0") #GPU에서 실행

In [11]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [12]:
# 입력과 출력을 저장(Hold)하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grade=False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 
# 계산할 필요가 없음을 나타냅니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [13]:
# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 
# 계산할 필요가 있음을 나타냅니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [14]:
learning_rate = 1e-6

In [15]:
for t in range(501):
    # 사용자 정의 함수를 적용하기 위해 Function.apply method를 사용합니다.
    # 이를 'relu'라고 이름(alias) 붙였습니다.
    relu = MyReLU.apply
    
    # 순전파 단계: Tensor 연산을 사용하여 y값을 예측합니다.
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산합니다.
    y_pred = relu(x.mm(w1)).mm(w2)
        
    # 손실(loss)를 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum()
    if t%20 == 0:
        print(t, loss.item())
    
    # autograde를 사용하여 역전파 단계를 계산합니다. 
    loss.backward()
    
    # 경사하강법(Gradient Descent)을 사용하여 가중치를 수동으로 갱신합니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다. 
        w1.grad.zero_()
        w2.grad.zero_()    

0 39911880.0
20 215712.921875
40 28315.1875
60 6184.56201171875
80 1684.270751953125
100 518.0689697265625
120 170.47726440429688
140 58.48392868041992
160 20.622568130493164
180 7.415342330932617
200 2.7177696228027344
220 1.0110408067703247
240 0.3797748386859894
260 0.143900066614151
280 0.05499255657196045
300 0.02125277929008007
320 0.008379928767681122
340 0.0034686941653490067
360 0.0015495000407099724
380 0.0007640960393473506
400 0.0004162409750279039
420 0.0002459258248563856
440 0.00015613512368872762
460 0.00010629875032464042
480 7.587280560983345e-05
500 5.684040661435574e-05


# TensorFlow: 정적 그래프(Static Graph)

PyTorch autograd는 Tensorflow와 많이 닮아보입니다. 두 프레임워크 모두 연산 그래프를 정의하며, 자동 미분을 사용하여 변화도를 계산합니다. 두 프레임워크의 가장 큰 차이점은 Tensorflow의 연산 그래프는 **정적**이며, PyTorch는 **동적** 연산 그래프를 사용한다는 것입니다.

Tensorflow에서는 연산 그래프를 한 번 정의한 후 동일한 그래프를 계속해서 실행하며 가능한 다른 입력 데이터를 전달합니다. PyTorch에서는 각각의 순전파 단계에서 새로운 연산 그래프를 정의합니다.

정적 그래프는 먼저(Up-front) 그래프를 최적화할 수 있기 때문에 좋습니다. 예를 들어 프레임워크가 효율을 위해 일부 그래프 연산을 합치거나, 여러 GPU나 시스템(machine)에 그래프를 배포하는 전략을 제시할 수 있습니다. 만약 동일한 그래프를 계속 재사용하면, 같은 그래프가 반복되면서 비싼(Costly) 최적화 비용을 잠재적으로 상환할 수 있습니다.

정적 그래프와 동적 그래프는 제어 흐름(Control flow) 측면에서도 다릅니다. 어떤 모델에서 각 데이터 지점(Point)마다 다른 연산을 수행하고 싶을 수 있습니다. 예를 들어 순환 신경망에서 각각의 데이터 지점마다 서로 다른 횟수만큼 펼칠(Unroll) 수 있습니다. 이러한 펼침은 반복문(Loop)으로 구현할 수 있습니다. 정적 그래프에서 반복문은 그래프의 일부가 돼야 합니다. 이러한 이유에서 Tensorflow는 그래프 내에 반복문을 포함하기 위해 **tf.scan**과 같은 연산자를 제공합니다. 동적 그래프에서는 이러한 상황이 더 단순(Simple)해집니다. 각 예제에 대한 그래프를 즉석(on-the-fly)에서 작성하기 때문에, 일반적인 명령형(Imperative) 제어 흐름을 사용하여 각각의 입력에 따라 다른 계산을 수행할 수 있습니다.

위의 PyTorch autograd 예제와는 대조적으로, TensorFlow를 사용하여 간단한 2-계층 신경망을 구성하겠습니다.

In [16]:
import tensorflow as tf
import numpy as np

In [17]:
# 먼저 연산 그래프를 구성하겠습니다.

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

In [18]:
# 입력과 정답(target) 데이터를 위한 placeholder를 생성합니다. 이는 우리가 그래프를
# 실행할 때 실제 데이터로 채워질 것입니다.
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

In [19]:
# 가중치를 저장하기 위한 Variable을 생성하고 무작위 데이터로 초기화합니다.
# Tensorflow의 Variable은 그래프가 실행되는 동안 그 값이 유지됩니다.
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

In [20]:
# 순전파 단계: Tensorflow의 Tensor 연산을 사용하여 y 값을 예측합니다.
# 이 코드가 어떠한 수치 연산을 실제로 수행하지는 않는다는 것을 유의하세요
# 이 단계에서는 나중에 실행할 연산 그래프를 구성하기만 합니다.
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

In [21]:
# Tensorflow의 Tensor 연산을 사용하여 손실(loss)을 계산합니다.
loss = tf.reduce_sum((y-y_pred) ** 2.0)

In [22]:
# 손실에 따른 w1, w2의 변화도(Gradients)를 계산합니다.
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

In [23]:
# 경사하강법(Gradient Descent)을 사용하여 가중치를 갱신합니다. 실제로 가중치를
# 갱신하기 위해서는 그래프가 실행될 때 new_w1과 new_w2 계산(evaluate)해야 합니다.
# Tensorflow에서 가중치의 값을 갱신하는 작업은 연산 그래프의 일부임을 유의하십시오.
# PyTorch에서는 이 작업이 연산 그래프의 밖에서 일어납니다.
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

In [24]:
# 지금까지 우리는 연산 그래프를 구성하였으므로, 실제로 그래프를 실행하기 위해 이제
# Tensorflow 세션(Session)에 들어가보겠습니다.
with tf.Session() as sess:
    # 그래프를 한 번 실행하여 Variable w1과 w2를 초기화합니다.
    sess.run(tf.global_variables_initializer())
    
    # 입력 데이터 x와 정답 데이터 y를 저장하기 위한 NumPy 배열을 생성합니다.
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    for t in range(501):
        # 그래프를 여러 번 실행합니다. 매번 그래프가 실행할 때마다 
        # feed_dict 인자(argument)로 명시하여 x_value를 x에, y_value를 y에 
        # 할당(bind)하고자 합니다. 또한, 그래프를 실행할 때마다 손실과 new_w1, new_w2 값을
        # 계산하려고 합니다. 이러한 Tensor들의 값은 NumPy 배열로 반환됩니다.
        loss_value, _, _ = sess.run([loss, new_w1, new_w2],
                                   feed_dict={x: x_value, y: y_value})
        if t%20 == 0:
            print(t, loss_value)

0 33415556.0
20 210055.19
40 27905.895
60 5659.0015
80 1382.3694
100 375.0264
120 109.41353
140 33.636364
160 10.76815
180 3.5589175
200 1.2056948
220 0.41666114
240 0.14629363
260 0.05203171
280 0.018745061
300 0.006938661
320 0.0027203052
340 0.0011731435
360 0.00056748005
380 0.00030668164
400 0.00018307929
420 0.00011747252
440 8.066726e-05
460 5.7960722e-05
480 4.402594e-05
500 3.4174656e-05
